In [42]:
### Import Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import pprint
from tabulate import tabulate
import warnings

%matplotlib inline
pd.set_option("display.max_columns",40)
warnings.filterwarnings("ignore")

from IPython.display import HTML


import keras
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingClassifier


from sklearn.metrics import confusion_matrix,accuracy_score,auc,roc_curve,f1_score
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler,binarize

from mlxtend.classifier import StackingClassifier
from xgboost import XGBClassifier
from vecstack import stacking
from pandas_ml import ConfusionMatrix

In [43]:
def get_roc_curve(model,x_test,y_test):
    prob=model.predict_proba(x_test)
    preds=prob[:,1]
    fpr, tpr, threshold = roc_curve(y_test, preds)
    roc_auc =auc(fpr, tpr)
    plt.figure(figsize=(10,8))
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr, index = i), '1-fpr' : pd.Series(1-fpr, index = i), 'tf' : pd.Series(tpr - (1-fpr), index = i), 'thresholds' : pd.Series(threshold, index = i)})
    threshold=roc.ix[(roc.tf-0).abs().argsort()[:1]]['thresholds'].values[0]
    print("Optimal Threshold is",threshold)
    y_pred_class=binarize(preds.reshape(1,-1),threshold)[0]
    accuracy=accuracy_score(y_test,y_pred_class)
    print("Accuracy of Logistic Model on training data is:",accuracy)
    cm=ConfusionMatrix(y_test,y_pred_class)
    display(cm)
    return cm,accuracy,threshold,roc_auc

In [44]:
# # ## Read the csv file which contains the data aggregated on user level 
# # z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_0.csv',index_col='Source_User')

# # z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
# #     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
# #     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# # ]


# # columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
# #     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
# #     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# # ]

# # print("The feature vector for infected users and sample of uninfected users")
# # print('Infected/Unifected Users')
# # display(z.Target.value_counts())
# # ## Create a feature set
# # X=z.iloc[:,z.columns!='Target']
# # display(X.head(2))
# # X=X.values

# # ### Build our target variable
# # print("1 -> Infected\n0 -> Uninfected")
# # Y=z.Target
# # Y=Y.apply(lambda x: 1 if x=='infected' else 0)
# # display(Y.head(10))
# # Y=Y.values

# # ### Scale feature set  using Standard Scaler
# # from sklearn.preprocessing import StandardScaler
# # scaler=StandardScaler()
# # scaled_X=scaler.fit_transform(X)
# # print("The train/test split ratio is 80:20")
# # from sklearn.model_selection import train_test_split
# # X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)


# # clf1 = BaggingClassifier(n_estimators=300, max_samples=0.6,max_features=0.4,
# #                       bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)
# # clf2 =  GradientBoostingClassifier(loss='deviance', learning_rate=0.05, criterion='mse', min_samples_split=0.3,
# #                                min_samples_leaf=20,subsample=0.5,random_state = 0, n_estimators = 150)
# # clf3 = AdaBoostClassifier(n_estimators=50,learning_rate=1.5,algorithm='SAMME',random_state=0)
# # lr = LogisticRegression()
# # sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
# #                           use_probas=True,
# #                           average_probas=True,
# #                           meta_classifier=lr)

# # print('3-fold cross validation:\n')

# # for clf, label in zip([clf1, clf2, clf3, sclf], 
# #                       ['KNN', 
# #                        'Random Forest', 
# #                        'Naive Bayes',
# #                        'StackingClassifier']):

# #     scores = cross_val_score(clf, X_train,Y_train,cv=3, scoring='accuracy')
# #     print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
# #           % (scores.mean(), scores.std(), label))

# # sclf.fit(X_train,Y_train)
# # acc_test=accuracy_score(Y_test,sclf.predict(X_test))
# # cm_test=ConfusionMatrix(Y_test,sclf.predict(X_test))
# # print(acc_test)
# # print(cm_test)

# # acc_train=sclf.score(X_train,Y_train)
# # cm_train=ConfusionMatrix(Y_train,sclf.predict(X_train))
# # print(acc_train)
# # print(cm_train)

# # cm,accuracy,threshold,roc_auc=get_roc_curve(sclf,X_test,Y_test)

# # t=pd.DataFrame([accuracy, cm.TP,      cm.TN,      cm.FP,      cm.FN,      cm.FPR,      cm.FNR, threshold,roc_auc]).transpose()
# # t.head()



# ## Read the csv file which contains the data aggregated on user level 
# z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_10.csv',index_col='Source_User')

# z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
#     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
#     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# ]


# columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
#     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
#     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# ]

# print("The feature vector for infected users and sample of uninfected users")
# print('Infected/Unifected Users')
# display(z.Target.value_counts())
# ## Create a feature set
# X=z.iloc[:,z.columns!='Target']
# display(X.head(2))
# X=X.values

# ### Build our target variable
# print("1 -> Infected\n0 -> Uninfected")
# Y=z.Target
# Y=Y.apply(lambda x: 1 if x=='infected' else 0)
# display(Y.head(10))
# Y=Y.values

# ### Scale feature set  using Standard Scaler
# from sklearn.preprocessing import StandardScaler
# scaler=StandardScaler()
# scaled_X=scaler.fit_transform(X)
# print("The train/test split ratio is 80:20")
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)


# clf1 = BaggingClassifier(n_estimators=300, max_samples=0.6,max_features=0.4,
#                       bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)
# clf2 =  GradientBoostingClassifier(loss='deviance', learning_rate=0.05, criterion='mse', min_samples_split=0.3,
#                                min_samples_leaf=20,subsample=0.5,random_state = 0, n_estimators = 150)
# clf3 = AdaBoostClassifier(n_estimators=50,learning_rate=1.5,algorithm='SAMME',random_state=0)
# lr = LogisticRegression()
# sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
#                           use_probas=True,
#                           average_probas=True,
#                           meta_classifier=lr)

# print('3-fold cross validation:\n')

# for clf, label in zip([clf1, clf2, clf3, sclf], 
#                       ['KNN', 
#                        'Random Forest', 
#                        'Naive Bayes',
#                        'StackingClassifier']):

#     scores = cross_val_score(clf, X_train,Y_train,cv=3, scoring='accuracy')
#     print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
#           % (scores.mean(), scores.std(), label))

# sclf.fit(X_train,Y_train)
# acc_test=accuracy_score(Y_test,sclf.predict(X_test))
# cm_test=ConfusionMatrix(Y_test,sclf.predict(X_test))
# print(acc_test)
# print(cm_test)

# acc_train=sclf.score(X_train,Y_train)
# cm_train=ConfusionMatrix(Y_train,sclf.predict(X_train))
# print(acc_train)
# print(cm_train)

# cm,accuracy,threshold,roc_auc=get_roc_curve(sclf,X_test,Y_test)

# t=pd.DataFrame([accuracy, cm.TP,      cm.TN,      cm.FP,      cm.FN,      cm.FPR,      cm.FNR, threshold,roc_auc]).transpose()
# t.head()







# ## Read the csv file which contains the data aggregated on user level 
# z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_20.csv',index_col='Source_User')

# z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
#     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
#     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# ]


# columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
#     '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
#     '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
# ]

# print("The feature vector for infected users and sample of uninfected users")
# print('Infected/Unifected Users')
# display(z.Target.value_counts())
# ## Create a feature set
# X=z.iloc[:,z.columns!='Target']
# display(X.head(2))
# X=X.values

# ### Build our target variable
# print("1 -> Infected\n0 -> Uninfected")
# Y=z.Target
# Y=Y.apply(lambda x: 1 if x=='infected' else 0)
# display(Y.head(10))
# Y=Y.values

# ### Scale feature set  using Standard Scaler
# from sklearn.preprocessing import StandardScaler
# scaler=StandardScaler()
# scaled_X=scaler.fit_transform(X)
# print("The train/test split ratio is 80:20")
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)


# clf1 = BaggingClassifier(n_estimators=300, max_samples=0.6,max_features=0.4,
#                       bootstrap=True,bootstrap_features=True,oob_score=True,random_state=0,n_jobs=-1)
# clf2 =  GradientBoostingClassifier(loss='deviance', learning_rate=0.05, criterion='mse', min_samples_split=0.3,
#                                min_samples_leaf=20,subsample=0.5,random_state = 0, n_estimators = 150)
# clf3 = AdaBoostClassifier(n_estimators=50,learning_rate=1.5,algorithm='SAMME',random_state=0)
# lr = LogisticRegression()
# sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
#                           use_probas=True,
#                           average_probas=True,
#                           meta_classifier=lr)

# print('3-fold cross validation:\n')

# for clf, label in zip([clf1, clf2, clf3, sclf],['KNN', 'Random Forest', 'Naive Bayes','StackingClassifier']):

#     scores = cross_val_score(clf, X_train,Y_train,cv=3, scoring='accuracy')
#     print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
#           % (scores.mean(), scores.std(), label))

# sclf.fit(X_train,Y_train)
# acc_test=accuracy_score(Y_test,sclf.predict(X_test))
# cm_test=ConfusionMatrix(Y_test,sclf.predict(X_test))
# print(acc_test)
# print(cm_test)

# acc_train=sclf.score(X_train,Y_train)
# cm_train=ConfusionMatrix(Y_train,sclf.predict(X_train))
# print(acc_train)
# print(cm_train)

# cm,accuracy,threshold,roc_auc=get_roc_curve(sclf,X_test,Y_test)

# t=pd.DataFrame([accuracy, cm.TP,      cm.TN,      cm.FP,      cm.FN,      cm.FPR,      cm.FNR, threshold,roc_auc]).transpose()
# t.head()

In [45]:
cric=['eee73a52c55dbef6c5b4ec7ec0cd3d200a90f528b7344b7337a546f7734774db',
       '1510909c55f8994397a5aced5728fca609aff1b6bab31080e919795aeb2db97d',
       'cff22bd1423ddbbbf00818b3d866cb4a5f4a8603ee607a824ad96224072f4076',
       '1ed642700929c99329b1ec5e6ddaa5b55db8ec04f776a690e4e22b86e89136db',
       'c96be1bc82e9c8d37ba23c7077d47f003a88fe0b5e0ef79ffa830d9552eeed1e',
       '61066be7a03d4212024695c0d434a3f6cf94f81d89911b6f5cd9c96819b0b064',
       'a2d8c624dbcb266d862fbbfb3920c06f051b4c78c8d8f033ccea74b92c433112',
       'f8bb4b13cb09963f5c4e51af61a1946eb2dbda71188d1e203d14163868ac5752',
       'cca0d130b4966f6bcc4e7093295e2637b48f421c120f4d6bfece4938e1c61ae7',
       'cf4dffb6fb12b9a95d1cc43717950e389db7e3c7c9e6c4a3d56d8e0c7a98891b',
       'bc87724fed1de64476927e18776a43eea899dea2e2a39ba6bf69dfe1eaf8b145',
       'db663ee7c30f6e366215d111f745205c73789819f5781028b573a197b0388e23',
       '5b2109fa7d9154dc8b15a1ecee9f520330519e9e4d8ea51a61e01cbc44600a86',
       '5672b291263e608577995d50cd51efd4662af2b9a3d327db7d6c264112bd7d57',
       '6d0095effd2435ab7c4557e92698c57345ac103d4668cf902221faec2052d1ea',
       '4121c37e4f4081f7e533940466b0962159f00840b14dca4479eb17e0d5807b15',
       '996116b7d5b47888082dd8caf8911ef352ebe6311210d813fbc18710e12a0091']

In [46]:
crit=['eee73a52c55dbef6c5b4ec7ec0cd3d200a90f528b7344b7337a546f7734774db',
       '1510909c55f8994397a5aced5728fca609aff1b6bab31080e919795aeb2db97d',
       'cff22bd1423ddbbbf00818b3d866cb4a5f4a8603ee607a824ad96224072f4076',
       '1ed642700929c99329b1ec5e6ddaa5b55db8ec04f776a690e4e22b86e89136db',
       'c96be1bc82e9c8d37ba23c7077d47f003a88fe0b5e0ef79ffa830d9552eeed1e',
       '61066be7a03d4212024695c0d434a3f6cf94f81d89911b6f5cd9c96819b0b064',
       'a2d8c624dbcb266d862fbbfb3920c06f051b4c78c8d8f033ccea74b92c433112',
       'f8bb4b13cb09963f5c4e51af61a1946eb2dbda71188d1e203d14163868ac5752',
       'cca0d130b4966f6bcc4e7093295e2637b48f421c120f4d6bfece4938e1c61ae7',
       'cf4dffb6fb12b9a95d1cc43717950e389db7e3c7c9e6c4a3d56d8e0c7a98891b',
       'bc87724fed1de64476927e18776a43eea899dea2e2a39ba6bf69dfe1eaf8b145',
       'db663ee7c30f6e366215d111f745205c73789819f5781028b573a197b0388e23',
       '5b2109fa7d9154dc8b15a1ecee9f520330519e9e4d8ea51a61e01cbc44600a86',
       '5672b291263e608577995d50cd51efd4662af2b9a3d327db7d6c264112bd7d57',
       '6d0095effd2435ab7c4557e92698c57345ac103d4668cf902221faec2052d1ea',
       '4121c37e4f4081f7e533940466b0962159f00840b14dca4479eb17e0d5807b15',
       '996116b7d5b47888082dd8caf8911ef352ebe6311210d813fbc18710e12a0091']

med=['07a805db92f77828144af02c110b4107381c8608ff69cbe8322b1619550dca26',
       'eee73a52c55dbef6c5b4ec7ec0cd3d200a90f528b7344b7337a546f7734774db',
       'cef0d1cb38af8ad44a4c03c5718984ae948de73fefefc7f16e8e87a27bd719ff',
       '1da09acf8ddc082a297329856e8512e0aa25eef1650894f3e39d24006b3b9900',
       '335e6cc2247d9cbd022e95cff5b6d3bd1da43b40cc993c2b840800394bb569b4',
       '98a76f9142fc866cdad367b1ea486dbb062fa0782e92e7eddf470385af2b5484',
       'b1d81f7de972e44a6845b9556ed3485a9b9a4a27c499d707ca91d3f5aa2334cc',
       '5c95fd730ce2612f44cb7a9c5802d9c5c9e90d7efdeb81026d8a89ab5a3c0a8a',
       '9861528d3962a11deb48775c63026bc8cbdf39aaf3f90aa0b87fef5445d5926c',
       'ce91776ab66403168967f46d8170b6949035822ba671e2ae1a4e5b012f71fc73',
       '42ee4c8133818830b3d084683b96a8209ca6b0d6112f25d0002681474301404c',
       'b0de67ec800462fd2105c2c142ae87db886e09c6eb0a1d33448afbf25fb57dfa',
       'f11e0185ff5a19066b53c1f397619e29796fd2f4623ac97a54764213cd997ccc',
       'd4f33d03ecd342a343c01d55c5b1191e37b1bc77452e20516b52cc6de0611f33',
       '26490d34b73ba281b2a586fdbfc9d13b48deaa204944733b859bf412f1fa9b0e',
       '2b9d80a899a0e34177f2b0ad451f18509d8b73540ce2b51fc2dc652495c99417',
       '95fef8a0d3377bb3fb41d589e5091eece852026be1b1224383265748863b6671',
       '0caa319489c018ddc7862bf2dafc054889ec8066de3f5b166d1a26e9f12920a4',
       '9bf55dee2eb78fca489c15f4c05644d7e259c413f4f97ab1d622e9999703ff7a',
       '94e77fdd5917ebf29fc743bf64798a5dd7c60068fcd029559e4e03afae2cda39',
       '8b7d9bb36cbb3b6d3defe024d4b980f5bedb71d8875456642e2b173dede2a3c7',
       'f71994f88f63bd783f50648de55e879dc8580e20c771703b852c0e3036667f46',
       'f52846069af3ef50f230f86ac40cdfe41e461a5e749951f112cd0f7c8e605a9e',
       '4677f4c9612858664db2eac9f788886ba53d3b48d79320c1fb15aaf04e00d970',
       '201b65b57be8702dfa4593b39f0d5a9d79662627dc885199c1373b188d4a4a47',
       'bb445f8ffb0724b780364707b75ade0df70a8ee22fd1a9f89af3f9d469928b3c',
       '989678db8131e49fd29adc46c2e5e1a6e9ab6823203b37954a24b2b68a98083e',
       '682eb13643f153128c9c47afe6be08554cd042d0b38815b6745a2ec45b472a70',
       '2587a851f09c2bb4d22b7daf8ec30af164ad94a7f5d1620d60d271b85fff6f64',
       '27b0bf1efe11e967f3ddb1b1c2760eaff1659bc221ae60ba772c972de59bf4c1',
       'bcca3c58857d6029838ff9d34f3c576d76ac8dbccbe85c39961f0a9a8abe99e9',
       '7a4da56250e41de51576ffbfbc5db2aa6b38cc35867a20b4d28318a6e2c1c813',
       '775b40dfdcf5074b07b4c6639073b185e1133e0aaf468ede9ffd5fab9dd9943f',
       '6f6d4c27d6351b6dd1209847771f8aeda37ec301b11b442620e32bbda346bcb7',
       'f55a241cd8a27c7392b8ab9049495758f6895fefa5538e8aca2c27a204ecb0c3',
       '705d59380dfcee0454a0d3d9063900bf5e6706ab9f642b68a00f53099af0eafd',
       'c9c33a60aaec935cd9d3d1420c46c0e4bcec43463aac55dd454705b04f6a9f8c',
       'a14cbfea9a24e5720c98ad25ccdde36970b5ede89f9ac783a4bbebb8b61a1568',
       '58d466231ab231ae49c7b315b995b8e1b4916c0c3e79fe3c057b06ab8c3d6ec8',
       '497982be6706563d82b4f49446534877235c8f7f85099592e79d4da57a26d761',
       '005030a1caa49429f288b79917630d9a991690b95dde910c8f00a6b926e68c31',
       '22a6f0f1cec1769ea1d0898f041c36ebbfa5b1bf6c6b14d9d11b9119d02ae6ea',
       '316af6d1233ba85aa8e83c4defad35dd37cdc3d753700f5d16bd9da4b6af6ccd',
       'c2b4a8f51a1f1eddc7a459bbc3911da76708a878fcd0a16cb5bb6d2ab3dfb5d0',
       '204fa067ef1d8b8b034acfa1693f8c53e86c84305ee7503a199f40d3722ee57e',
       'ed91f04fbd495e2d5a00f327756aa4882f8b9d4f5d58b444150f5d89b2befebc',
       'eed1b477ab451eb20e847c90b3752b5a7d6c83d3b0c36285450f6731add2063e',
       'dff5c1910184b240d839f5a61143d74c7e537b1a7a6104c1878b69475e224b67',
       '5f78d8ea5d220fd4371a9d5b67f9704172b7e95aa85756242ea58a09ad6a2189',
       '84a578c10ef1a342ad03312393548815acba50632dd302b60f96e273abab64c8',
       '54b356aa258f805ac028589e97883ab66d011a00dd9e2a409eb5579f4e81abd4',
       '8eba4debb95ef877faed7041269483166a3145a7a590cba966da0a5aa394f2f3',
       '571fdff9a66b4534942db0655d35eef2dde903259323d6a36128a00a48b94efa',
       '68e9f31a207a753cc3460b17afd291ef4972d57cb149dbdac093fc4c986ad994',
       '802f787a8d4ebe3d4830943c5f4d50572512c76002ffdc4f9acaf238da85fb36',
       '645ee32ae1c84817505808b1a84e53565542cebd89d00d56207c3a8d111c60e5',
       '421548d50c73674295d827ff088f1d353441fdd64a0fa7ff74863cd8113bf991',
       'adbaa75ca91a86b069ec2810a47b226803cfcc90fb537c09a6945d9a13f19845',
       'a905175974a71113bb8794ec07b0e03add816a329c9e971c833710d4bdd52abf',
       'b85725fc8c972f70e3e8a290fc32b75df44bff1fbf31555ff512074003b6387b',
       '376ffc521ffbf5d3cb640e186b9db7a98795f7656a82b73207f6884ca1194589',
       'a6d0045c1c7ffb7db8fdcbe34d7ea0f46ce12606114b63b3702d312be308f9de']

In [47]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_0.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bb98d45a9bd0c40362a3ec37503c10b8238194752d355742cd3ea35221dc98b1,4607,77.507923,40.245713,100.283265,6930.568483,89947.788800,3901.0,113.0,58,86,1116,1646.597569,167,295,328,198,223,135,144,145,194,197,196,178,120,145,118,224,172,241,246,151,165,177,134,314
36d9d88841607ef5b1614c98854aadf57ca7de84854e90e215137c4ad8c46fc5,169,5.840237,5.597633,149.763314,1016.976331,3222.147929,168.0,4.0,9,11,16,40005.674556,0,0,0,0,0,0,0,0,8,8,0,0,39,48,40,10,4,4,2,6,0,0,0,0


In [48]:
def get_stats(a):
    x=0
    y=0
    for i in med:
        if i in m:
            x=x+a[a['Source_User']==i][0].values
            y=y+a[a['Source_User']==i]['Target'].values
    print("For high Severity:")
    print("identified:",x)
    print("Actual",y)
    print("For critical Severity")
    x=0
    y=0
    for i in cric:
        if i in m:
            x=x+a[a['Source_User']==i][0].values
            y=y+a[a['Source_User']==i]['Target'].values
    print("identified:",x)
    print("Actual",y)

## 80:20 Split

### random State 0

In [49]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_0.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bb98d45a9bd0c40362a3ec37503c10b8238194752d355742cd3ea35221dc98b1,4607,77.507923,40.245713,100.283265,6930.568483,89947.788800,3901.0,113.0,58,86,1116,1646.597569,167,295,328,198,223,135,144,145,194,197,196,178,120,145,118,224,172,241,246,151,165,177,134,314
36d9d88841607ef5b1614c98854aadf57ca7de84854e90e215137c4ad8c46fc5,169,5.840237,5.597633,149.763314,1016.976331,3222.147929,168.0,4.0,9,11,16,40005.674556,0,0,0,0,0,0,0,0,8,8,0,0,39,48,40,10,4,4,2,6,0,0,0,0


In [50]:
### Build our target variable
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
bb98d45a9bd0c40362a3ec37503c10b8238194752d355742cd3ea35221dc98b1    1
36d9d88841607ef5b1614c98854aadf57ca7de84854e90e215137c4ad8c46fc5    0
56a003f863f8cc9342216fa01a3ab0aebf4a04ed0dd8d5b89a95f779cc707dd5    1
f894aec69ce612e29505d6d3cd9112bbe39d6a46e7da2682d14a1cbd9c4cd297    0
9aadb359b9763e3d68e62ad3c406245fcf0bc4919395dbf42532e49d8b5aa02d    1
dbea3804991397c518e0198f2b63ecab35cbf38083cb5e19bb9d4b4f4bacaa2d    1
f2327a8577d940bf7100353fce41bbde184c6e4ead193ab60986bb3ef2551718    1
d815eb7ef3af2a56e8260b0b3a6bb3c1bf70c625619caa930cff27beced41ea1    1
ae3fe786080d0c4a0d3520152bb59d60836d2ae6fdb4870a1879b98ea7ec02fc    1
45b5cf89d8e8ff70ce05570a97d5ff05783283281eade7d8b736c94a24cfb16c    1
Name: Target, dtype: int64

In [51]:
### Scale feature set  using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)
print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)

The train/test split ratio is 80:20


### GBC

In [52]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.1, criterion='mse', min_samples_split=0.2,
                               min_samples_leaf=10,subsample=0.7,random_state = 0, n_estimators = 100)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [10]
Actual [12]
For critical Severity
identified: [0]
Actual [2]


### Adaboost

In [53]:
ada=AdaBoostClassifier(n_estimators=150,learning_rate=0.5,algorithm='SAMME.R',random_state=0)
ada.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(ada.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values

get_stats(a)

For high Severity:
identified: [10]
Actual [12]
For critical Severity
identified: [0]
Actual [2]


### Random State 10

In [54]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_10.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
644acf067fe760509d55b90a2ab91e9c5e78d260c7214a3449af4e82e30f9f87,3030,230.950495,80.524092,84.155116,9686.861716,287727.528383,2741.0,11.0,40,63,820,2395.224422,195,256,110,55,63,53,56,31,79,81,39,75,98,81,159,264,181,142,177,135,120,184,240,156
a434cc3ad58dedb0ed629a1bf82cb810fdad9a7ab47ce851fca5724939971862,599,158.008347,57.856427,82.028381,6161.512521,198461.532554,587.0,12.0,30,70,210,12098.198664,0,3,3,2,4,2,0,34,70,92,117,30,10,14,7,5,25,37,61,27,27,25,1,3


In [55]:
### Build our target variable
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
644acf067fe760509d55b90a2ab91e9c5e78d260c7214a3449af4e82e30f9f87    0
a434cc3ad58dedb0ed629a1bf82cb810fdad9a7ab47ce851fca5724939971862    1
06fe39332e5261ed8a5f1bb9d62559deb0de1620963ebbded14eecf78b8a0a20    0
5455eddbb8ee814c41a27e8676a6cc34faa6fc295c6e92588fc75dbe7f61369b    0
47a99c45d555e42658dbf94be7417ec6c327ecfc05ed5c57eaac1de3bd43bb86    1
60ff3ff13bb2e6a16dc866bc314964cc6d60b1e9891d805e7d9e99b1864ae896    1
a4244338cc086f69de152063f5b11482be2d007a5da089331dc2fe778e459e26    1
70de2ff22d8dd1433aa38947470f94f455bcbdabb1789e81332ba5c390408acc    1
6c49763f6a507ebf7aff12e40a61cfe3624fca5d5ac9dc1b86ac38c68ca4b1ba    0
ab60d8ea5924263ddfde9661ee4a4081f95679c77492da4479cd0350f56c7de3    0
Name: Target, dtype: int64

In [56]:
### Scale feature set  using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)
print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)

The train/test split ratio is 80:20


### GBC

In [57]:
gbc=GradientBoostingClassifier(loss='deviance', learning_rate=0.03, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=20,subsample=0.7,random_state = 0, n_estimators = 100)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [12]
Actual [14]
For critical Severity
identified: [2]
Actual [3]


### Adaboost

In [58]:
ada=AdaBoostClassifier(n_estimators=150,learning_rate=1.5,algorithm='SAMME',random_state=0)
ada.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(ada.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values

get_stats(a)

For high Severity:
identified: [11]
Actual [14]
For critical Severity
identified: [2]
Actual [3]


### Random State 20

In [59]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_20.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
b8517ecc923cf25374ba60a6da4f97be55f9f01896a2d641329bc603046dfcc4,651,35.807988,35.328725,108.442396,3976.233487,37072.858679,639.0,8.0,21,23,158,11131.820276,6,0,2,0,0,0,0,13,8,34,95,39,76,71,74,66,114,27,14,0,0,7,3,2
307c636564c5ec9897e5062605e2982daf781f0f82b0234f2cbc55919f8d04e2,4059,321.478197,131.395910,73.695245,31586.025868,413326.700419,3627.0,18.0,40,31,974,1803.300813,196,310,532,308,202,101,45,281,95,38,93,95,206,102,104,84,194,144,116,161,158,138,159,197


In [60]:
### Build our target variable
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
b8517ecc923cf25374ba60a6da4f97be55f9f01896a2d641329bc603046dfcc4    0
307c636564c5ec9897e5062605e2982daf781f0f82b0234f2cbc55919f8d04e2    0
11e8b7b973c885d9d2560baeec4673bc6b13aab08d9e49bc9c519727595e4d3e    1
41851f48304067368da8f12794e2f539330cc7ef6732c1440ed4c5eb06b9cc5d    0
08402d90e55ea64677880c21fc3850422ad2e08f730282d271027f0fea60b4b6    1
952aed50327c6ac4e19a033bb585959b33ea2ca897f985dc33a1bb6a1f3d7cc7    0
6b42f8e7f92d2fccc0fae12167ef5e929f394e87e9c70bb36ac160bf6d6cb0f4    0
c9140a85217daf0d18148d489fa6d3293cbe499584b12e28be0e381a20eeb737    1
647e8a6690cd21f7bae287782ec6a621cae3a5ba6f37b779d99f28a7851b1bfc    0
42d06643547a80b9343fa1a09ca32b0c3bb84caa31bfb45c629b6835072e364b    0
Name: Target, dtype: int64

In [61]:
### Scale feature set  using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)
print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)

The train/test split ratio is 80:20


### GBC

In [62]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.05, criterion='mse', min_samples_split=0.2,
                               min_samples_leaf=5,subsample=0.5,random_state = 0, n_estimators = 50)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [6]
Actual [8]
For critical Severity
identified: [2]
Actual [3]


### Adaboost

In [63]:
ada=AdaBoostClassifier(n_estimators=150,learning_rate=0.5,algorithm='SAMME',random_state=0)
ada.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(ada.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values

get_stats(a)

For high Severity:
identified: [6]
Actual [8]
For critical Severity
identified: [2]
Actual [3]


## 70:30 Split

### Random state 0

In [64]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_0.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bb98d45a9bd0c40362a3ec37503c10b8238194752d355742cd3ea35221dc98b1,4607,77.507923,40.245713,100.283265,6930.568483,89947.788800,3901.0,113.0,58,86,1116,1646.597569,167,295,328,198,223,135,144,145,194,197,196,178,120,145,118,224,172,241,246,151,165,177,134,314
36d9d88841607ef5b1614c98854aadf57ca7de84854e90e215137c4ad8c46fc5,169,5.840237,5.597633,149.763314,1016.976331,3222.147929,168.0,4.0,9,11,16,40005.674556,0,0,0,0,0,0,0,0,8,8,0,0,39,48,40,10,4,4,2,6,0,0,0,0


In [65]:
### Build our target variable
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
bb98d45a9bd0c40362a3ec37503c10b8238194752d355742cd3ea35221dc98b1    1
36d9d88841607ef5b1614c98854aadf57ca7de84854e90e215137c4ad8c46fc5    0
56a003f863f8cc9342216fa01a3ab0aebf4a04ed0dd8d5b89a95f779cc707dd5    1
f894aec69ce612e29505d6d3cd9112bbe39d6a46e7da2682d14a1cbd9c4cd297    0
9aadb359b9763e3d68e62ad3c406245fcf0bc4919395dbf42532e49d8b5aa02d    1
dbea3804991397c518e0198f2b63ecab35cbf38083cb5e19bb9d4b4f4bacaa2d    1
f2327a8577d940bf7100353fce41bbde184c6e4ead193ab60986bb3ef2551718    1
d815eb7ef3af2a56e8260b0b3a6bb3c1bf70c625619caa930cff27beced41ea1    1
ae3fe786080d0c4a0d3520152bb59d60836d2ae6fdb4870a1879b98ea7ec02fc    1
45b5cf89d8e8ff70ce05570a97d5ff05783283281eade7d8b736c94a24cfb16c    1
Name: Target, dtype: int64

In [66]:
### Scale feature set  using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)
print("The train/test split ratio is 70:30")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.3)

The train/test split ratio is 70:30


### Extra Tree Classifier

In [67]:
ext_tree=ExtraTreeClassifier(criterion='gini',max_depth=10,max_features='auto',min_impurity_decrease=1e-3,
                             min_samples_leaf=5,min_samples_split=2,splitter='best',random_state=0)

ext_tree.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(ext_tree.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [12]
Actual [18]
For critical Severity
identified: [1]
Actual [6]


### GBC

In [68]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.1, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=20,subsample=0.7,random_state = 0, n_estimators = 50)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [13]
Actual [18]
For critical Severity
identified: [1]
Actual [6]


### Random State 10

In [69]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_10.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
644acf067fe760509d55b90a2ab91e9c5e78d260c7214a3449af4e82e30f9f87,3030,230.950495,80.524092,84.155116,9686.861716,287727.528383,2741.0,11.0,40,63,820,2395.224422,195,256,110,55,63,53,56,31,79,81,39,75,98,81,159,264,181,142,177,135,120,184,240,156
a434cc3ad58dedb0ed629a1bf82cb810fdad9a7ab47ce851fca5724939971862,599,158.008347,57.856427,82.028381,6161.512521,198461.532554,587.0,12.0,30,70,210,12098.198664,0,3,3,2,4,2,0,34,70,92,117,30,10,14,7,5,25,37,61,27,27,25,1,3


### Extra Tree Classifier

In [70]:
ext_tree=ExtraTreeClassifier(criterion='gini',max_depth=10,max_features='auto',min_impurity_decrease=1e-3,
                             min_samples_leaf=2,min_samples_split=2,splitter='best',random_state=0)

ext_tree.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(ext_tree.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [12]
Actual [18]
For critical Severity
identified: [1]
Actual [6]


### GBC

In [71]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.03, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=5,subsample=0.5,random_state = 0, n_estimators = 100)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [13]
Actual [18]
For critical Severity
identified: [1]
Actual [6]


### Random State 20

In [72]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_20.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
b8517ecc923cf25374ba60a6da4f97be55f9f01896a2d641329bc603046dfcc4,651,35.807988,35.328725,108.442396,3976.233487,37072.858679,639.0,8.0,21,23,158,11131.820276,6,0,2,0,0,0,0,13,8,34,95,39,76,71,74,66,114,27,14,0,0,7,3,2
307c636564c5ec9897e5062605e2982daf781f0f82b0234f2cbc55919f8d04e2,4059,321.478197,131.395910,73.695245,31586.025868,413326.700419,3627.0,18.0,40,31,974,1803.300813,196,310,532,308,202,101,45,281,95,38,93,95,206,102,104,84,194,144,116,161,158,138,159,197


In [73]:
### Build our target variable
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
b8517ecc923cf25374ba60a6da4f97be55f9f01896a2d641329bc603046dfcc4    0
307c636564c5ec9897e5062605e2982daf781f0f82b0234f2cbc55919f8d04e2    0
11e8b7b973c885d9d2560baeec4673bc6b13aab08d9e49bc9c519727595e4d3e    1
41851f48304067368da8f12794e2f539330cc7ef6732c1440ed4c5eb06b9cc5d    0
08402d90e55ea64677880c21fc3850422ad2e08f730282d271027f0fea60b4b6    1
952aed50327c6ac4e19a033bb585959b33ea2ca897f985dc33a1bb6a1f3d7cc7    0
6b42f8e7f92d2fccc0fae12167ef5e929f394e87e9c70bb36ac160bf6d6cb0f4    0
c9140a85217daf0d18148d489fa6d3293cbe499584b12e28be0e381a20eeb737    1
647e8a6690cd21f7bae287782ec6a621cae3a5ba6f37b779d99f28a7851b1bfc    0
42d06643547a80b9343fa1a09ca32b0c3bb84caa31bfb45c629b6835072e364b    0
Name: Target, dtype: int64

In [74]:
### Scale feature set  using Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)
print("The train/test split ratio is 70:30")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.3)

The train/test split ratio is 70:30


### Extra Tree Classifier

In [75]:
ext_tree=ExtraTreeClassifier(criterion='entropy',max_depth=20,max_features='auto',min_impurity_decrease=1e-3,
                             min_samples_leaf=2,min_samples_split=5,splitter='best',random_state=0)

ext_tree.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(ext_tree.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [10]
Actual [14]
For critical Severity
identified: [2]
Actual [6]


### GBC

In [76]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.05, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=5,subsample=1,random_state = 0, n_estimators = 150)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [11]
Actual [14]
For critical Severity
identified: [2]
Actual [6]


### 80:20 with PCA

#### Random State 0

In [77]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_0.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bb98d45a9bd0c40362a3ec37503c10b8238194752d355742cd3ea35221dc98b1,4607,77.507923,40.245713,100.283265,6930.568483,89947.788800,3901.0,113.0,58,86,1116,1646.597569,167,295,328,198,223,135,144,145,194,197,196,178,120,145,118,224,172,241,246,151,165,177,134,314
36d9d88841607ef5b1614c98854aadf57ca7de84854e90e215137c4ad8c46fc5,169,5.840237,5.597633,149.763314,1016.976331,3222.147929,168.0,4.0,9,11,16,40005.674556,0,0,0,0,0,0,0,0,8,8,0,0,39,48,40,10,4,4,2,6,0,0,0,0


In [78]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
bb98d45a9bd0c40362a3ec37503c10b8238194752d355742cd3ea35221dc98b1    1
36d9d88841607ef5b1614c98854aadf57ca7de84854e90e215137c4ad8c46fc5    0
56a003f863f8cc9342216fa01a3ab0aebf4a04ed0dd8d5b89a95f779cc707dd5    1
f894aec69ce612e29505d6d3cd9112bbe39d6a46e7da2682d14a1cbd9c4cd297    0
9aadb359b9763e3d68e62ad3c406245fcf0bc4919395dbf42532e49d8b5aa02d    1
dbea3804991397c518e0198f2b63ecab35cbf38083cb5e19bb9d4b4f4bacaa2d    1
f2327a8577d940bf7100353fce41bbde184c6e4ead193ab60986bb3ef2551718    1
d815eb7ef3af2a56e8260b0b3a6bb3c1bf70c625619caa930cff27beced41ea1    1
ae3fe786080d0c4a0d3520152bb59d60836d2ae6fdb4870a1879b98ea7ec02fc    1
45b5cf89d8e8ff70ce05570a97d5ff05783283281eade7d8b736c94a24cfb16c    1
Name: Target, dtype: int64

In [79]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.2)

Variance explained using 13 components is: 0.922348229056
The train/test split ratio is 80:20


### Logistic 

In [80]:
logistic=LogisticRegression(C=0.5,dual=False,fit_intercept=False,intercept_scaling=0.5,max_iter=50,multi_class='multinomial',
                           penalty='l2',solver='sag',tol=0.001)
logistic.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(logistic.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [10]
Actual [12]
For critical Severity
identified: [0]
Actual [2]


### Random Forest

In [81]:
rf=RandomForestClassifier()
rf.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(rf.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [7]
Actual [12]
For critical Severity
identified: [0]
Actual [2]


### GBC

In [82]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.05, criterion='mse', min_samples_split=0.3,
                               min_samples_leaf=20,subsample=1,random_state = 0, n_estimators = 150)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [9]
Actual [12]
For critical Severity
identified: [0]
Actual [2]


### ETC

In [83]:
ext_tree=ExtraTreeClassifier(criterion='gini',max_depth=10,max_features='auto',min_impurity_decrease=1e-2,
                             min_samples_leaf=2,min_samples_split=2,splitter='best',random_state=0)

ext_tree.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(ext_tree.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [10]
Actual [12]
For critical Severity
identified: [0]
Actual [2]


### NN2

In [84]:
def build_keras_classifier(optimizer):
    deep_learning=Sequential()
    deep_learning.add(Dense(units=64,activation='relu',use_bias=True,kernel_initializer='uniform',input_dim=13))
    deep_learning.add(Dense(units=128,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=256,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=512,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=1,activation='sigmoid',use_bias=True,kernel_initializer='uniform'))
    deep_learning.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return deep_learning
classifier=KerasClassifier(build_fn=build_keras_classifier,batch_size=20,nb_epoch=100,verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
parmas= {'batch_size':[20],'nb_epoch':[120],'optimizer':['adam','rmsprop']}
grid_search=GridSearchCV(estimator=classifier,param_grid=parmas,scoring='accuracy',cv=kfold,verbose=0)
grid_search.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(grid_search.predict(X_test).reshape(1,len(X_test))[0])
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [9]
Actual [12]
For critical Severity
identified: [0]
Actual [2]


### Random State 10

In [85]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_10.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
644acf067fe760509d55b90a2ab91e9c5e78d260c7214a3449af4e82e30f9f87,3030,230.950495,80.524092,84.155116,9686.861716,287727.528383,2741.0,11.0,40,63,820,2395.224422,195,256,110,55,63,53,56,31,79,81,39,75,98,81,159,264,181,142,177,135,120,184,240,156
a434cc3ad58dedb0ed629a1bf82cb810fdad9a7ab47ce851fca5724939971862,599,158.008347,57.856427,82.028381,6161.512521,198461.532554,587.0,12.0,30,70,210,12098.198664,0,3,3,2,4,2,0,34,70,92,117,30,10,14,7,5,25,37,61,27,27,25,1,3


In [86]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
644acf067fe760509d55b90a2ab91e9c5e78d260c7214a3449af4e82e30f9f87    0
a434cc3ad58dedb0ed629a1bf82cb810fdad9a7ab47ce851fca5724939971862    1
06fe39332e5261ed8a5f1bb9d62559deb0de1620963ebbded14eecf78b8a0a20    0
5455eddbb8ee814c41a27e8676a6cc34faa6fc295c6e92588fc75dbe7f61369b    0
47a99c45d555e42658dbf94be7417ec6c327ecfc05ed5c57eaac1de3bd43bb86    1
60ff3ff13bb2e6a16dc866bc314964cc6d60b1e9891d805e7d9e99b1864ae896    1
a4244338cc086f69de152063f5b11482be2d007a5da089331dc2fe778e459e26    1
70de2ff22d8dd1433aa38947470f94f455bcbdabb1789e81332ba5c390408acc    1
6c49763f6a507ebf7aff12e40a61cfe3624fca5d5ac9dc1b86ac38c68ca4b1ba    0
ab60d8ea5924263ddfde9661ee4a4081f95679c77492da4479cd0350f56c7de3    0
Name: Target, dtype: int64

In [87]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.2)

Variance explained using 13 components is: 0.915617452326
The train/test split ratio is 80:20


### Logistic 

In [88]:
logistic=LogisticRegression(C=0.5,dual=False,fit_intercept=False,intercept_scaling=0.5,max_iter=50,multi_class='multinomial',
                           penalty='l2',solver='newton-cg',tol=0.00001)
logistic.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(logistic.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [12]
Actual [14]
For critical Severity
identified: [3]
Actual [3]


### Random Forest

In [89]:
rf=RandomForestClassifier()
rf.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(rf.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [8]
Actual [14]
For critical Severity
identified: [2]
Actual [3]


### GBC

In [90]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.05, criterion='mse', min_samples_split=0.2,
                               min_samples_leaf=15,subsample=0.5,random_state = 0, n_estimators = 150)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [11]
Actual [14]
For critical Severity
identified: [2]
Actual [3]


### ETC

In [91]:
ext_tree=ExtraTreeClassifier(criterion='entropy',max_depth=20,max_features='auto',min_impurity_decrease=1e-2,
                             min_samples_leaf=5,min_samples_split=2,splitter='best',random_state=0)

ext_tree.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(ext_tree.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [12]
Actual [14]
For critical Severity
identified: [3]
Actual [3]


### NN2

In [92]:
def build_keras_classifier(optimizer):
    deep_learning=Sequential()
    deep_learning.add(Dense(units=64,activation='relu',use_bias=True,kernel_initializer='uniform',input_dim=13))
    deep_learning.add(Dense(units=128,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=256,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=512,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=1,activation='sigmoid',use_bias=True,kernel_initializer='uniform'))
    deep_learning.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return deep_learning
classifier=KerasClassifier(build_fn=build_keras_classifier,batch_size=20,nb_epoch=100,verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
parmas= {'batch_size':[20],'nb_epoch':[120],'optimizer':['adam','rmsprop']}
grid_search=GridSearchCV(estimator=classifier,param_grid=parmas,scoring='accuracy',cv=kfold,verbose=0)
grid_search.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(grid_search.predict(X_test).reshape(1,len(X_test))[0])
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [10]
Actual [14]
For critical Severity
identified: [2]
Actual [3]


### Random State 20

In [93]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_20.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
b8517ecc923cf25374ba60a6da4f97be55f9f01896a2d641329bc603046dfcc4,651,35.807988,35.328725,108.442396,3976.233487,37072.858679,639.0,8.0,21,23,158,11131.820276,6,0,2,0,0,0,0,13,8,34,95,39,76,71,74,66,114,27,14,0,0,7,3,2
307c636564c5ec9897e5062605e2982daf781f0f82b0234f2cbc55919f8d04e2,4059,321.478197,131.395910,73.695245,31586.025868,413326.700419,3627.0,18.0,40,31,974,1803.300813,196,310,532,308,202,101,45,281,95,38,93,95,206,102,104,84,194,144,116,161,158,138,159,197


In [94]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
b8517ecc923cf25374ba60a6da4f97be55f9f01896a2d641329bc603046dfcc4    0
307c636564c5ec9897e5062605e2982daf781f0f82b0234f2cbc55919f8d04e2    0
11e8b7b973c885d9d2560baeec4673bc6b13aab08d9e49bc9c519727595e4d3e    1
41851f48304067368da8f12794e2f539330cc7ef6732c1440ed4c5eb06b9cc5d    0
08402d90e55ea64677880c21fc3850422ad2e08f730282d271027f0fea60b4b6    1
952aed50327c6ac4e19a033bb585959b33ea2ca897f985dc33a1bb6a1f3d7cc7    0
6b42f8e7f92d2fccc0fae12167ef5e929f394e87e9c70bb36ac160bf6d6cb0f4    0
c9140a85217daf0d18148d489fa6d3293cbe499584b12e28be0e381a20eeb737    1
647e8a6690cd21f7bae287782ec6a621cae3a5ba6f37b779d99f28a7851b1bfc    0
42d06643547a80b9343fa1a09ca32b0c3bb84caa31bfb45c629b6835072e364b    0
Name: Target, dtype: int64

In [95]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 80:20")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.2)

Variance explained using 13 components is: 0.920535156414
The train/test split ratio is 80:20


### Logistic 

In [96]:
logistic=LogisticRegression(C=0.5,dual=False,fit_intercept=False,intercept_scaling=0.5,max_iter=50,multi_class='multinomial',
                           penalty='l2',solver='newton-cg',tol=0.00001)
logistic.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(logistic.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [6]
Actual [8]
For critical Severity
identified: [2]
Actual [3]


### Random Forest

In [97]:
rf=RandomForestClassifier()
rf.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(rf.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [4]
Actual [8]
For critical Severity
identified: [2]
Actual [3]


### GBC

In [98]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.05, criterion='mse', min_samples_split=0.2,
                               min_samples_leaf=20,subsample=0.7,random_state = 0, n_estimators = 150)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [6]
Actual [8]
For critical Severity
identified: [2]
Actual [3]


### ETC

In [99]:
ext_tree=ExtraTreeClassifier(criterion='entropy',max_depth=20,max_features='auto',min_impurity_decrease=1e-2,
                             min_samples_leaf=3,min_samples_split=2,splitter='best',random_state=0)

ext_tree.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(ext_tree.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [6]
Actual [8]
For critical Severity
identified: [2]
Actual [3]


### NN2

In [100]:
def build_keras_classifier(optimizer):
    deep_learning=Sequential()
    deep_learning.add(Dense(units=64,activation='relu',use_bias=True,kernel_initializer='uniform',input_dim=13))
    deep_learning.add(Dense(units=128,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=256,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=512,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=1,activation='sigmoid',use_bias=True,kernel_initializer='uniform'))
    deep_learning.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return deep_learning
classifier=KerasClassifier(build_fn=build_keras_classifier,batch_size=20,nb_epoch=100,verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
parmas= {'batch_size':[20],'nb_epoch':[120],'optimizer':['adam','rmsprop']}
grid_search=GridSearchCV(estimator=classifier,param_grid=parmas,scoring='accuracy',cv=kfold,verbose=0)
grid_search.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(grid_search.predict(X_test).reshape(1,len(X_test))[0])
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [6]
Actual [8]
For critical Severity
identified: [2]
Actual [3]


### 70:30 Split with PCA

#### Random State 0

In [101]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_0.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bb98d45a9bd0c40362a3ec37503c10b8238194752d355742cd3ea35221dc98b1,4607,77.507923,40.245713,100.283265,6930.568483,89947.788800,3901.0,113.0,58,86,1116,1646.597569,167,295,328,198,223,135,144,145,194,197,196,178,120,145,118,224,172,241,246,151,165,177,134,314
36d9d88841607ef5b1614c98854aadf57ca7de84854e90e215137c4ad8c46fc5,169,5.840237,5.597633,149.763314,1016.976331,3222.147929,168.0,4.0,9,11,16,40005.674556,0,0,0,0,0,0,0,0,8,8,0,0,39,48,40,10,4,4,2,6,0,0,0,0


In [102]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
bb98d45a9bd0c40362a3ec37503c10b8238194752d355742cd3ea35221dc98b1    1
36d9d88841607ef5b1614c98854aadf57ca7de84854e90e215137c4ad8c46fc5    0
56a003f863f8cc9342216fa01a3ab0aebf4a04ed0dd8d5b89a95f779cc707dd5    1
f894aec69ce612e29505d6d3cd9112bbe39d6a46e7da2682d14a1cbd9c4cd297    0
9aadb359b9763e3d68e62ad3c406245fcf0bc4919395dbf42532e49d8b5aa02d    1
dbea3804991397c518e0198f2b63ecab35cbf38083cb5e19bb9d4b4f4bacaa2d    1
f2327a8577d940bf7100353fce41bbde184c6e4ead193ab60986bb3ef2551718    1
d815eb7ef3af2a56e8260b0b3a6bb3c1bf70c625619caa930cff27beced41ea1    1
ae3fe786080d0c4a0d3520152bb59d60836d2ae6fdb4870a1879b98ea7ec02fc    1
45b5cf89d8e8ff70ce05570a97d5ff05783283281eade7d8b736c94a24cfb16c    1
Name: Target, dtype: int64

In [103]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 70:30")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.3)

Variance explained using 13 components is: 0.922348229056
The train/test split ratio is 70:30


### Logistic 

In [104]:
logistic=LogisticRegression(C=0.5,dual=False,fit_intercept=False,intercept_scaling=0.5,max_iter=50,multi_class='multinomial',
                           penalty='l2',solver='sag',tol=0.001)
logistic.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(logistic.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [14]
Actual [18]
For critical Severity
identified: [2]
Actual [6]


### Random Forest

In [105]:
rf=RandomForestClassifier()
rf.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(rf.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [9]
Actual [18]
For critical Severity
identified: [0]
Actual [6]


### GBC

In [106]:
gbc=GradientBoostingClassifier(loss='deviance', learning_rate=0.05, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=10,subsample=0.7,random_state = 0, n_estimators = 50)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [12]
Actual [18]
For critical Severity
identified: [1]
Actual [6]


### NN2

In [107]:
def build_keras_classifier(optimizer):
    deep_learning=Sequential()
    deep_learning.add(Dense(units=64,activation='relu',use_bias=True,kernel_initializer='uniform',input_dim=13))
    deep_learning.add(Dense(units=128,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=256,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=512,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=1,activation='sigmoid',use_bias=True,kernel_initializer='uniform'))
    deep_learning.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return deep_learning
classifier=KerasClassifier(build_fn=build_keras_classifier,batch_size=20,nb_epoch=100,verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
parmas= {'batch_size':[20],'nb_epoch':[120],'optimizer':['adam','rmsprop']}
grid_search=GridSearchCV(estimator=classifier,param_grid=parmas,scoring='accuracy',cv=kfold,verbose=0)
grid_search.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(grid_search.predict(X_test).reshape(1,len(X_test))[0])
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [11]
Actual [18]
For critical Severity
identified: [2]
Actual [6]


### Random State 10

In [108]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_10.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
644acf067fe760509d55b90a2ab91e9c5e78d260c7214a3449af4e82e30f9f87,3030,230.950495,80.524092,84.155116,9686.861716,287727.528383,2741.0,11.0,40,63,820,2395.224422,195,256,110,55,63,53,56,31,79,81,39,75,98,81,159,264,181,142,177,135,120,184,240,156
a434cc3ad58dedb0ed629a1bf82cb810fdad9a7ab47ce851fca5724939971862,599,158.008347,57.856427,82.028381,6161.512521,198461.532554,587.0,12.0,30,70,210,12098.198664,0,3,3,2,4,2,0,34,70,92,117,30,10,14,7,5,25,37,61,27,27,25,1,3


In [109]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
644acf067fe760509d55b90a2ab91e9c5e78d260c7214a3449af4e82e30f9f87    0
a434cc3ad58dedb0ed629a1bf82cb810fdad9a7ab47ce851fca5724939971862    1
06fe39332e5261ed8a5f1bb9d62559deb0de1620963ebbded14eecf78b8a0a20    0
5455eddbb8ee814c41a27e8676a6cc34faa6fc295c6e92588fc75dbe7f61369b    0
47a99c45d555e42658dbf94be7417ec6c327ecfc05ed5c57eaac1de3bd43bb86    1
60ff3ff13bb2e6a16dc866bc314964cc6d60b1e9891d805e7d9e99b1864ae896    1
a4244338cc086f69de152063f5b11482be2d007a5da089331dc2fe778e459e26    1
70de2ff22d8dd1433aa38947470f94f455bcbdabb1789e81332ba5c390408acc    1
6c49763f6a507ebf7aff12e40a61cfe3624fca5d5ac9dc1b86ac38c68ca4b1ba    0
ab60d8ea5924263ddfde9661ee4a4081f95679c77492da4479cd0350f56c7de3    0
Name: Target, dtype: int64

In [110]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 70:30")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.3)

Variance explained using 13 components is: 0.915617452326
The train/test split ratio is 70:30


### Logistic 

In [111]:
logistic=LogisticRegression(C=0.5,dual=False,fit_intercept=False,intercept_scaling=0.5,max_iter=50,multi_class='multinomial',
                           penalty='l2',solver='sag',tol=0.00001)
logistic.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(logistic.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [17]
Actual [21]
For critical Severity
identified: [3]
Actual [3]


### Random Forest

In [112]:
rf=RandomForestClassifier()
rf.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(rf.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [14]
Actual [21]
For critical Severity
identified: [1]
Actual [3]


### GBC

In [113]:
gbc=GradientBoostingClassifier(loss='deviance', learning_rate=0.05, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=20,subsample=0.7,random_state = 0, n_estimators = 150)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [15]
Actual [21]
For critical Severity
identified: [3]
Actual [3]


### NN2

In [114]:
def build_keras_classifier(optimizer):
    deep_learning=Sequential()
    deep_learning.add(Dense(units=64,activation='relu',use_bias=True,kernel_initializer='uniform',input_dim=13))
    deep_learning.add(Dense(units=128,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=256,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=512,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=1,activation='sigmoid',use_bias=True,kernel_initializer='uniform'))
    deep_learning.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return deep_learning
classifier=KerasClassifier(build_fn=build_keras_classifier,batch_size=20,nb_epoch=100,verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
parmas= {'batch_size':[20],'nb_epoch':[120],'optimizer':['adam','rmsprop']}
grid_search=GridSearchCV(estimator=classifier,param_grid=parmas,scoring='accuracy',cv=kfold,verbose=0)
grid_search.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(grid_search.predict(X_test).reshape(1,len(X_test))[0])
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [17]
Actual [21]
For critical Severity
identified: [2]
Actual [3]


### Random State 20

In [115]:
## Read the csv file which contains the data aggregated on user level 
z=pd.read_csv('../Collecting logs and Building Feature Set/Feature_Vector_70_30_2_15_random_state_20.csv',index_col='Source_User')

z.columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','Target','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]


columns=['NREC','PAR','PAS','LEN','BYS','BYR','SP','DP','NAPP','SIP','DIP','TDIF',
    '0000','0100','0200','0300','0400','0500','0600','0700','0800','0900','1000','1100','1200',
    '1300','1400','1500','1600','1700','1800','1900','2000','2100','2200','2300'
]

print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())
## Create a feature set
X=z.iloc[:,z.columns!='Target']
display(X.head(2))
X=X.values

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


uninfected    5789
infected      2481
Name: Target, dtype: int64

,NREC,PAR,PAS,LEN,BYS,BYR,SP,DP,NAPP,SIP,DIP,TDIF,0000,0100,0200,0300,0400,0500,0600,0700,0800,0900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
Source_User,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
b8517ecc923cf25374ba60a6da4f97be55f9f01896a2d641329bc603046dfcc4,651,35.807988,35.328725,108.442396,3976.233487,37072.858679,639.0,8.0,21,23,158,11131.820276,6,0,2,0,0,0,0,13,8,34,95,39,76,71,74,66,114,27,14,0,0,7,3,2
307c636564c5ec9897e5062605e2982daf781f0f82b0234f2cbc55919f8d04e2,4059,321.478197,131.395910,73.695245,31586.025868,413326.700419,3627.0,18.0,40,31,974,1803.300813,196,310,532,308,202,101,45,281,95,38,93,95,206,102,104,84,194,144,116,161,158,138,159,197


In [116]:
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(10))
# Y=Y.values

1 -> Infected
0 -> Uninfected


Source_User
b8517ecc923cf25374ba60a6da4f97be55f9f01896a2d641329bc603046dfcc4    0
307c636564c5ec9897e5062605e2982daf781f0f82b0234f2cbc55919f8d04e2    0
11e8b7b973c885d9d2560baeec4673bc6b13aab08d9e49bc9c519727595e4d3e    1
41851f48304067368da8f12794e2f539330cc7ef6732c1440ed4c5eb06b9cc5d    0
08402d90e55ea64677880c21fc3850422ad2e08f730282d271027f0fea60b4b6    1
952aed50327c6ac4e19a033bb585959b33ea2ca897f985dc33a1bb6a1f3d7cc7    0
6b42f8e7f92d2fccc0fae12167ef5e929f394e87e9c70bb36ac160bf6d6cb0f4    0
c9140a85217daf0d18148d489fa6d3293cbe499584b12e28be0e381a20eeb737    1
647e8a6690cd21f7bae287782ec6a621cae3a5ba6f37b779d99f28a7851b1bfc    0
42d06643547a80b9343fa1a09ca32b0c3bb84caa31bfb45c629b6835072e364b    0
Name: Target, dtype: int64

In [117]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)

pca=PCA(n_components=13,whiten=True,random_state=0)
X_pca_13=pca.fit_transform(scaled_X)
print("Variance explained using 13 components is:",sum(pca.explained_variance_ratio_))

print("The train/test split ratio is 70:30")
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_pca_13,Y,random_state=0,test_size=0.3)

Variance explained using 13 components is: 0.920535156414
The train/test split ratio is 70:30


### Logistic 

In [118]:
logistic=LogisticRegression(C=0.5,dual=False,fit_intercept=False,intercept_scaling=0.5,max_iter=50,multi_class='multinomial',
                           penalty='l2',solver='newton-cg',tol=0.00001)
logistic.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(logistic.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [11]
Actual [14]
For critical Severity
identified: [2]
Actual [6]


### Random Forest

In [119]:
rf=RandomForestClassifier()
rf.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(rf.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [8]
Actual [14]
For critical Severity
identified: [2]
Actual [6]


### GBC

In [120]:
gbc=GradientBoostingClassifier(loss='exponential', learning_rate=0.05, criterion='mse', min_samples_split=0.1,
                               min_samples_leaf=10,subsample=0.7,random_state = 0, n_estimators = 100)

gbc.fit(X_train,Y_train)

train=pd.DataFrame(Y_test)
test=pd.DataFrame(gbc.predict(X_test))
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [11]
Actual [14]
For critical Severity
identified: [2]
Actual [6]


### NN2

In [121]:
def build_keras_classifier(optimizer):
    deep_learning=Sequential()
    deep_learning.add(Dense(units=64,activation='relu',use_bias=True,kernel_initializer='uniform',input_dim=13))
    deep_learning.add(Dense(units=128,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=256,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=512,activation='relu',use_bias=True,kernel_initializer='uniform'))
    deep_learning.add(Dense(units=1,activation='sigmoid',use_bias=True,kernel_initializer='uniform'))
    deep_learning.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return deep_learning
classifier=KerasClassifier(build_fn=build_keras_classifier,batch_size=20,nb_epoch=100,verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
parmas= {'batch_size':[20],'nb_epoch':[120],'optimizer':['adam','rmsprop']}
grid_search=GridSearchCV(estimator=classifier,param_grid=parmas,scoring='accuracy',cv=kfold,verbose=0)
grid_search.fit(X_train,Y_train)
train=pd.DataFrame(Y_test)
test=pd.DataFrame(grid_search.predict(X_test).reshape(1,len(X_test))[0])
test.set_index(train.index.get_values(),inplace=True)
a=train.join(test,how='inner').reset_index()
m=a.Source_User.values
get_stats(a)

For high Severity:
identified: [12]
Actual [14]
For critical Severity
identified: [2]
Actual [6]


In [122]:
### Original Creator : Darshan Bhansali
### HTML code to hide the input cells 
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To toggle code, click <a href="javascript:code_toggle()">here</a>.''')